去水印模型冲刺赛

# 百度网盘AI大赛-图像处理挑战赛：去水印模型冲刺赛

# 一、比赛介绍

为了帮助人们消除带有水印的图片日常生活中带有水印的图片很常见，即使是PS专家，也很难快速且不留痕迹的去除水印。在上次比赛中我们举办了去水印模型大赛，本次我们提高了难度，提高了在水印种类的广泛性，同时在模型性能上做了更高的要求，向各位选手发起挑战。

# 二、评价标准

- 客观评价指标为 PSNR 和 MSSSIM；
- 用于评价的机器环境仅提供两种框架模型运行环境：paddlepaddle 和 onnxruntime，其他框架模型可转换为上述两种框架的模型；
- 机器配置：V100，显存15G，内存10G；
- 单张图片耗时>1.2s，决赛中的性能分数记0分。

因此，应尽可能不能使用过大的模型。

# 三、改进和优化的之处
本方案对比baseline有很多优化之处：
-对训练数据集做数据增强，对测试数据也做了一定的增强
- 选取背景色彩复杂、前景水印易混淆导致的恢复效果较差的难样本，专门进行 finetune；
- 使用多模集成（如果不考量参数）、每次热重启时新增可训练参数等；
- 在对比学习损失上探索其他 encoder 及其不同层的 embedding 输出、使用更多的 negative samples 等；
- 探索模型压缩相关技术（剪枝、量化、蒸馏等）；
- 探索使用在线添加更丰富多样的水印。

# 四、网络构造

- 本项目所设计的网络基于 PMRID UNet，对水印图像进行像素级回归预测和转换。其核心在于使用了层次化和逆瓶颈的深度可分离卷积操作，在确保模型大小和推理速度符合要求的前提下，有效改善了模型容量不足所带来的精度问题。架构如下所示（来源：https://github.com/yuezewang/2022-MegCup-RawImageDenoising/blob/main/main.py
）：

![](https://ai-studio-static-online.cdn.bcebos.com/9592decbd46140d691ef5aafe2dff447dd4dce67e8e0485daa69bb772f4a93b2)










 同时，本项目提供了最终的模型参数文件 **model.pdiparams, model.pdiparams.info, model.pdmodel** 用于复现。

#### 数据解压

- 执行如下命令解压本项目压缩好的部分数据集及其标签到 Notebook 所在目录：

In [ ]:
# 创建目录 - 若目录已存在则无需运行
! mkdir targets  # 创建标签目录
! mkdir samples  # 创建样本目录

In [ ]:
# 解压 标签/目标/背景图片 到 target 目录下

In [ ]:
# 检查
! cd targets ; tree

In [ ]:
# 解压第 8 部分训练集到 sample 目录下
! tar -xvf data/data141714/watermark_datasets.part8.tar -C ./samples

In [ ]:
# 检查
! cd samples ; tree

#### 依赖导入与参数设置

- 调试数据和模型时，令 **DEBUG=True** 以使用很少数据测验；使用全数据训练正常训练时，令 **DEBUG=False**。
- 通过设置 **STAGE** 为 **[0, 1, 2, 3, 4]** 中的整数来决定当前训练阶段及其部分超参数。若在当前的部分数据集环境下运行，请令 **STAGE=0** 用于示范；若在本地的全数据集环境下实施训练，则 **STAGE 不再包含 0**。
- 手动设置 **MODEL_LOAD_PATH** 的路径将指定模型用于 STAGE 为 **[2, 3, 4]** 阶段的重启训练。例如，设 STAGE=1 时，**验证 Score 最高** 的模型为 model_stage1_epoch8_step500.pdparams，则 STAGE=2 时，令 MODEL_LOAD_PATH = './model_stage1_epoch8_step500.pdparams' 即可在其基础上渐进优化。
- 为提升性能，可选取当前 STAGE 的 **3-5** 个模型权重实施 **EMA** (详见 **ema.py**) 来融合模型，再用于下一 STAGE 训练。
- 在 **STAGE=4** 时，默认使用的 L1Loss 也能得到较好的结果。若要使用 **对比学习损失** ，请下载 **[VGG19权重链接](https://pan.baidu.com/s/19wJRGS-n7e_LktwHBpkG5w?pwd=b1xq)** 并设法放置到当前目录下 (AIStudio 限制 150MB 上传文件大小)，然后保持 CLLOSS=True。

In [ ]:
# -------------------------------------------------------------------------------------------------------------------
# 依赖导入
# -------------------------------------------------------------------------------------------------------------------

import gc
import os
import sys
import time
import math
import glob
import random

import cv2
import numpy as np
import pandas as pd
from PIL import Image

import paddle
import paddle.nn as nn
import paddle.nn.functional as F

from network import Net
from metrics import ContrastLoss, MS_SSIM, PSNR

import warnings 
warnings.filterwarnings('ignore')

if paddle.is_compiled_with_cuda():
    paddle.set_device('gpu:0')
else:
    paddle.set_device('cpu')
    
# -------------------------------------------------------------------------------------------------------------------
# 部分数据集下的全局参数与路径设置（全数据集的设置详见 train.py）
# -------------------------------------------------------------------------------------------------------------------

# DEBUG = True 则使用很少样本实施 Debug 试验；DEBUG = False 则使用给定数据实施正常训练 ================================
DEBUG = False  # True #  

DEBUG_TRAIN_SAMPLES = 16  	 # 用于 Debug 试验的训练集样本数
DEBUG_VALID_SAMPLES = 16  	 # 用于 Debug 试验的验证集样本数

BG_SAMPLES = 1448         	 # 训练集背景样本索引划分阈值 (Target)
VALID_SAMPLES_INDEX = 11  	 # 验证集样本采样间隔

# 实施 4 阶段渐进训练, STAGE 为不同训练阶段标志
STAGE = 0  # 1, 2, 3, 4

# 第 4 阶段 (精细微调阶段)
if STAGE == 4:
    BS = 1        		     # 训练集批大小 Batch Size
    LR = 5e-5     		     # 学习率 Learn Rate
    EPOCH = 1     		     # 训练轮数 Epoch
    SIZE = 896    		     # 训练集图像尺寸 Image Size
    TRAIN_SAMPLES_INDEX = 1  # 训练集集样本采样间隔
    CLLOSS = True
    PRINT = 4000  		     # 验证迭代步数间隔
    
# 第 3 阶段
elif STAGE == 3:	
    BS = 2
    LR = 1e-4
    EPOCH = 2
    SIZE = 768 
    TRAIN_SAMPLES_INDEX = 2  
    CLLOSS = False
    PRINT = 2000
	
# 第 2 阶段
elif STAGE == 2:
    BS = 4
    LR = 5e-4
    EPOCH = 4
    SIZE = 640
    TRAIN_SAMPLES_INDEX = 4 
    CLLOSS = False
    PRINT = 1000

# 第 1 阶段 (粗放收敛阶段)
elif STAGE == 1:
    BS = 8
    LR = 1e-3
    EPOCH = 8
    SIZE = 512
    TRAIN_SAMPLES_INDEX = 8
    CLLOSS = False
    PRINT = 500

# 第 0 阶段 (本项目 Notebook 用于部分数据集的示范阶段 =====================================================================)
else:
    BS = 8
    LR = 1e-3
    EPOCH = 2
    SIZE = 512
    TRAIN_SAMPLES_INDEX = 1
    CLLOSS = False
    PRINT = 100

print(f"=======================> stage {STAGE} <=======================")

# 固定随机数种子
SEED = 42
paddle.seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

# 训练集路径
TRAIN_SAMPLES_PATH = "./samples/"  		     # 训练样本图片路径
TRAIN_TARGETS_PATH = "./targets/bg_images/"  # 训练标签图片路径

BUILD_SAMPLES_DF = True # False  # 只需执行一次, 生成 df_all.csv 后改成 False 即可
ALL_SAMPLES_DF_PATH = './df_all.csv'
TRAIN_SAMPLES_DF_PATH = f"./df_train_stage{STAGE}.csv"  # 训练样本 DataFrame 路径
VALID_SAMPLES_DF_PATH = f"./df_valid_stage{STAGE}.csv"  # 验证样本 DataFrame 路径

MODEL_LOAD_PATH = ''  # 用于 finetune 的前一 stage 模型路径

# 五、模型训练

1. 训练时数据增强
- 水平翻转：为避免模型过度拟合具有相同水印模式（特别是方向），在训练时实施 50% 概率的水平翻转。
- 循环填充：使用循环填充确保输入样本及其目标图片的最短边边长不小于预设尺寸，同时保持水印模式一致。
- 随机裁剪：为避免放缩失真和显存不足，并增强模型对不同图像局部的泛化能力，按预设尺寸随机裁剪样本及其目标。

![](https://ai-studio-static-online.cdn.bcebos.com/011e6e7509e844c4892bddfac090e2345afd344cccea46c3ae24d23a88c16402)


2. 训练验证集处理

- 训练集：将背景图片编号小于 1600 的所有水印图像作为候选训练集，通过调整等距抽样间隔获取子集作为不同阶段的训练集。
- 验证集：将候选训练集外的所有水印图像作为候选验证集。综合验证效果、时间和显存占用，通过等距抽样获取大小约 2000 的子集作为验证集，同时令 50% 的图像水平翻转，以便于衡量模型对不同水印模式的繁花能力，与训练时的水平翻转相呼应。

```
class MyDataset(paddle.io.Dataset):
    def __init__(self, df, train=False):
        super(MyDataset, self).__init__()

        self.df = df
        self.train = train  # 训练/验证模式下, 采用不同的数据预处理策略
        print(f"nums data set: {len(self.df)}")

    def __getitem__(self, index):

        # 读取训练样本及其标签
        image = cv2.imread(self.df.iloc[index]['sample'])
        label = cv2.imread(self.df.iloc[index]['target'])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = cv2.cvtColor(label, cv2.COLOR_BGR2RGB)

        # 归一化
        image = image / 255.
        label = label / 255.

        # 训练时数据预处理策略
        if self.train == True:
            h, w, c = image.shape

            # 50% 概率训练样本水平翻转, 增强模型对不同 pattern 的泛化性能
            if np.random.randint(0, 2) == 1:
                image = paddle.vision.transforms.hflip(image)
                label = paddle.vision.transforms.hflip(label)

            # H, W, C -> C, H, W
            image = image.transpose((2,0,1))
            label = label.transpose((2,0,1))

            # C, H, W -> 1, C, H, W   &    np.ndarray -> paddle.tensor
            image = paddle.to_tensor(image[np.newaxis, ...], dtype='float32')
            label = paddle.to_tensor(label[np.newaxis, ...], dtype='float32')

            # 将训练样本及其标签循环 padding 至最短边不小于 SIZE
            pad = nn.Pad2D(padding=[0, max(SIZE-w, 0), 0, max(SIZE-h, 0)], mode='circular')  # padding [pad_left, pad_right, pad_top, pad_bottom]
            image = pad(image)
            label = pad(label)

            # 随机裁剪尺寸为 (SIZE, SIZE) 的 patch 用于训练
            h_new = np.random.randint(0, image.shape[-2]-SIZE+1)
            w_new = np.random.randint(0, image.shape[-1]-SIZE+1)
            image = image[..., h_new:h_new+SIZE, w_new:w_new+SIZE].squeeze(0)
            label = label[..., h_new:h_new+SIZE, w_new:w_new+SIZE].squeeze(0)

        # 验证时数据预处理策略
        else:
            # 50% 验证样本水平翻转, 考察模型对不同 pattern 的泛化性能
            if index % 2 == 0:
                image = paddle.vision.transforms.hflip(image)
                label = paddle.vision.transforms.hflip(label)

            # H, W, C -> C, H, W
            image = image.transpose((2,0,1))
            label = label.transpose((2,0,1))

            # np.ndarray -> paddle.tensor
            image = paddle.to_tensor(image, dtype='float32')
            label = paddle.to_tensor(label, dtype='float32')

        return image, label

    def __len__(self):
        return len(self.df)
```

3. 基于验证 Score 的指数移动平均

- 由于单模鲁棒性不足，且常规指数移动平均 (Exponential Moving Average, EMA) 根据时间远近对模型参数指数加权，缺乏针对性。为此，可以使用基于验证 Score 的指数移动平均 (Validation Score based EMA) 来对 3-5 个前一阶段的最佳模型参数实施指数加权，从而有针对性地提升模型效果。

```
def EMA(model, ema_model_path, model_path_list):

    ema_model = copy.deepcopy(model)
    ema_n = 0

    with paddle.no_grad():
        for _ckpt in model_path_list:
            model.load_dict(paddle.load(_ckpt))  # , map_location=torch.device('cpu')
            tmp_para_dict = dict(model.named_parameters())
            alpha = 1. / (ema_n + 1.)
            for name, para in ema_model.named_parameters():
                new_para = tmp_para_dict[name].clone() * alpha + para.clone() * (1. - alpha)
                para.set_value(new_para.clone())
            ema_n += 1

    paddle.save(ema_model.state_dict(), ema_model_path)
    print('ema finished !!!')

    return ema_model
```


4. 多阶段热重启渐进收敛

- 用大量相似数据训练小模型易导致过拟合，且受限于图像尺寸等超参数，直接单次端到端训练不易兼顾数据利用和模型收敛。可以通过多阶段训练实现模型渐进收敛，即：每阶段微调上一阶段的最佳模型，渐进重启学习率、增大图像尺寸、减少训练周期和批大小，从而令模型由粗到精地学习和收敛。

![](https://ai-studio-static-online.cdn.bcebos.com/77de3d2fb8e14b5f945aed69967b0709f55f615cf1354984a2ee625dbbc25910)

5. 对比学习损失

- 通过计算对比学习损失，在基于 VGG19 的特征嵌入空间中，缩小模型预测结果 (Anchor) 与作为目标的背景图片 (Positive) 的特征嵌入的 L1 距离，拉大模型预测结果 (Anchor) 与输入水印图像 (Negative) 的特征嵌入的 L1 距离。

- 设 Batch size 为 N，使用 VGG19 的 M 层特征嵌入 Φ(.) ，每次计算使用 1 个目标图片 (Positive) GT、1 个重建输出 (Anchor) O、K 个具有相同背景的水印图片 (Negative) O_Neg，则对比学习损失 L_CL 可表示为：

![](https://ai-studio-static-online.cdn.bcebos.com/f2a6d31a782b4d6dba483a0479779afc12f1f99bb8224ca585114b81ad37c42e)

```
class ContrastLoss(nn.Layer):
    def __init__(self, ablation=False):
        super(ContrastLoss, self).__init__()

        self.vgg = VGGExtractor(pretrained="./vgg.pdparams")  # VGG feature extractor
        self.l1 = paddle.nn.loss.L1Loss()  # L1Loss
        self.weights = [1.0/32, 1.0/16, 1.0/8, 1.0/4, 1.0]  # weights of features at different layers
        self.ab = ablation

    def forward(self, a, p, n):
        # anchor samples, positive samples, negative samples
        a_vgg, p_vgg, n_vgg = self.vgg(a), self.vgg(p), self.vgg(n)
        loss = 0
        d_ap, d_an = 0, 0
        for i in range(len(a_vgg)):
            d_ap = self.l1(a_vgg[i], p_vgg[i].detach())
            if not self.ab:
                d_an = self.l1(a_vgg[i], n_vgg[i].detach())
                contrastive = d_ap / (d_an + 1e-7)
            else:
                contrastive = d_ap
            loss += self.weights[i] * contrastive
        return loss
```


#### 训练验证集划分 

- 在部分训练集 part8 下，背景图片索引小于 BG_SAMPLES = 1448 的作为训练集，其余部分等距抽样作为验证集：

- 在完全训练集下，背景图片索引小于 BG_SAMPLES = 1600 的作为训练集，其余部分等距抽样作为验证集：

In [ ]:
# ---------------------------------------------------------------------------------------------------------------------------------
# 训练-验证集划分、采样与加载
# ---------------------------------------------------------------------------------------------------------------------------------

if BUILD_SAMPLES_DF:
    data = pd.DataFrame({'sample': glob.glob(TRAIN_SAMPLES_PATH + "*/*")})
    data['target'] = data['sample'].apply(lambda x: TRAIN_TARGETS_PATH + x.split('/')[-1][:14] + ".jpg")
    data['idx'] = data['sample'].apply(lambda x: x.split('/')[-1][9:14])
    data = data.sort_values(by=['target']).reset_index(drop=True)
    data = data.drop(data.tail(1).index)
    data.to_csv(ALL_SAMPLES_DF_PATH, index=False)
    data_nums = len(data) 
    print(data_nums)  # part8：101254

    data = pd.read_csv(ALL_SAMPLES_DF_PATH)
    print(data.iloc[0]['idx'], data.iloc[-1]['idx'])  # 1288  1472

    sample_train = []
    sample_valid = []
    for i in range(len(data)):
        cur = data.iloc[i]['idx']
        if (cur >= BG_SAMPLES) and i % VALID_SAMPLES_INDEX == 0:
            sample_valid.append(data.iloc[i]['sample'])
            # print(len(sample_valid))
        elif (cur < BG_SAMPLES) and i % TRAIN_SAMPLES_INDEX == 0:
            sample_train.append(data.iloc[i]['sample'])

    df_train = pd.DataFrame()
    df_train['sample'] = sample_train
    df_train['target'] = df_train['sample'].apply(lambda x: TRAIN_TARGETS_PATH + x.split('/')[-1][:14] + ".jpg")
    df_train = df_train.sort_values(by=['target']).reset_index(drop=True)
    df_train.to_csv(TRAIN_SAMPLES_DF_PATH, index=False)

    df_valid = pd.DataFrame()
    df_valid['sample'] = sample_valid
    df_valid['target'] = df_valid['sample'].apply(lambda x: TRAIN_TARGETS_PATH + x.split('/')[-1][:14] + ".jpg")
    df_valid = df_valid.sort_values(by=['target']).reset_index(drop=True)
    df_valid.to_csv(VALID_SAMPLES_DF_PATH, index=False)


# 加载训练集 DataFrame
df_train = pd.read_csv(TRAIN_SAMPLES_DF_PATH)
TRAIN_SAMPLES_NUMS = len(df_train)
print(TRAIN_SAMPLES_NUMS)

# 加载验证集 DataFrame
df_valid = pd.read_csv(VALID_SAMPLES_DF_PATH)
VALID_SAMPLES_NUMS  = len(df_valid)
print(VALID_SAMPLES_NUMS)

#### 功能函数定义

In [ ]:
# ---------------------------------------------------------------------------------------------------------------------------------
# 数据集定义与数据预处理
# ---------------------------------------------------------------------------------------------------------------------------------

class MyDataset(paddle.io.Dataset):
    def __init__(self, df, train=False):
        super(MyDataset, self).__init__()

        self.df = df
        self.train = train  # 训练/验证模式下, 采用不同的数据预处理策略
        print(f"nums data set: {len(self.df)}")

    def __getitem__(self, index):

        # 读取训练样本及其标签
        image = cv2.imread(self.df.iloc[index]['sample'])
        label = cv2.imread(self.df.iloc[index]['target'])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = cv2.cvtColor(label, cv2.COLOR_BGR2RGB)

        # 归一化
        image = image / 255.
        label = label / 255.

        # 训练时数据预处理策略
        if self.train == True:
            h, w, c = image.shape

            # 50% 概率训练样本水平翻转, 增强模型对不同 pattern 的泛化性能
            if np.random.randint(0, 2) == 1:
                image = paddle.vision.transforms.hflip(image)
                label = paddle.vision.transforms.hflip(label)

            # H, W, C -> C, H, W
            image = image.transpose((2,0,1))
            label = label.transpose((2,0,1))

            # C, H, W -> 1, C, H, W   &    np.ndarray -> paddle.tensor
            image = paddle.to_tensor(image[np.newaxis, ...], dtype='float32')
            label = paddle.to_tensor(label[np.newaxis, ...], dtype='float32')

            # 将训练样本及其标签循环 padding 至最短边不小于 SIZE
            pad = nn.Pad2D(padding=[0, max(SIZE-w, 0), 0, max(SIZE-h, 0)], mode='circular')  # padding [pad_left, pad_right, pad_top, pad_bottom]
            image = pad(image)
            label = pad(label)

            # 随机裁剪尺寸为 (SIZE, SIZE) 的 patch 用于训练
            h_new = np.random.randint(0, image.shape[-2]-SIZE+1)
            w_new = np.random.randint(0, image.shape[-1]-SIZE+1)
            image = image[..., h_new:h_new+SIZE, w_new:w_new+SIZE].squeeze(0)
            label = label[..., h_new:h_new+SIZE, w_new:w_new+SIZE].squeeze(0)

        # 验证时数据预处理策略
        else:
            # 50% 验证样本水平翻转, 考察模型对不同 pattern 的泛化性能
            if index % 2 == 0:
                image = paddle.vision.transforms.hflip(image)
                label = paddle.vision.transforms.hflip(label)

            # H, W, C -> C, H, W
            image = image.transpose((2,0,1))
            label = label.transpose((2,0,1))

            # np.ndarray -> paddle.tensor
            image = paddle.to_tensor(image, dtype='float32')
            label = paddle.to_tensor(label, dtype='float32')

        return image, label

    def __len__(self):
        return len(self.df)

# ---------------------------------------------------------------------------------------------------------------------------------
# 验证函数定义
# ---------------------------------------------------------------------------------------------------------------------------------

def valid(valid_loader, model, psnr_compute, ssim_compute):
    print('validating...')
    model.eval()
    
    psnrs = []
    ssims = []
    scores = []

    for step, (sample, target) in enumerate(valid_loader):
		# 原输入样本 shape 为 (n, c, h, w)
        n, c, h, w = sample.shape
        
        # 将所有图片的尺寸/边长循环 padding 为 SIZE 的倍数
        num_patch_h = (h // SIZE) + 1
        num_patch_w = (w // SIZE) + 1 

        pad = nn.Pad2D(padding=[0, num_patch_w*SIZE-w, 0, num_patch_h*SIZE-h], mode='circular')  # [pad_left, pad_right, pad_top, pad_bottom]
        sample_pad = pad(sample)
 
        # 等距切分为若干尺寸为 (SIZE, SIZE) 的 patch 并分别预测后拼接
        output = paddle.zeros_like(sample_pad)
        with paddle.no_grad():
            for i in range(num_patch_h):
                for j in range(num_patch_w):
					# 当前用于预测的 patch 切片
                    sample_pad_patch = sample_pad[..., i*SIZE: (i+1)*SIZE, j*SIZE: (j+1)*SIZE].clone()
                    # 训练期间，验证操作并不实施水平翻转自集成，以促使模型对不同水印 pattern 都能具有较好的恢复效果。
                    # 换言之，避免模型对某一水印 pattern 过拟合，从而主导性能变化并轻视对其他水印 pattern 的去除学习。
                    output[..., i*SIZE: (i+1)*SIZE, j*SIZE: (j+1)*SIZE] = model(sample_pad_patch)
                    
        # 切除不需要的预测结果, 还原尺寸为 (h, w) 的原输入样本
        output = output[..., :h, :w]

        # 逐样本计算 PSNR、SSIM 和 Score
        for i in range(len(sample)):
            a, b = output[i: i+1, ...] , target[i: i+1, ...]
            psnr = psnr_compute(a, b).item()
            ssim = ssim_compute(a, b).item()
            score = (psnr / 100 + ssim) / 2
            psnrs.append(psnr)
            ssims.append(ssim)
            scores.append(score)
            
        gc.collect()
        paddle.device.cuda.empty_cache()

    return np.mean(scores), np.mean(psnrs), np.mean(ssims)

#### 训练主函数定义

In [ ]:
# ---------------------------------------------------------------------------------------------------------------------------------
# 主函数定义
# ---------------------------------------------------------------------------------------------------------------------------------

def process():
    # 模型实例化
    model = Net()
    # 加载预/已训练模型 finetune
    if MODEL_LOAD_PATH:
        param_dict = paddle.load(MODEL_LOAD_PATH)
        model.load_dict(param_dict)
    model.train()

    # 数据集及其加载器实例化
    train_dataset = MyDataset(df_train[:DEBUG_TRAIN_SAMPLES], train=True) if DEBUG else MyDataset(df_train, train=True)
    valid_dataset = MyDataset(df_valid[:DEBUG_VALID_SAMPLES], train=False) if DEBUG else MyDataset(df_valid, train=False)

    train_loader = paddle.io.DataLoader(
        train_dataset,
        batch_size=BS,
        num_workers=0,  # 若机器不错可改为 8
        shuffle=True,
        drop_last=True
    )

    valid_loader = paddle.io.DataLoader(
        valid_dataset,
        batch_size=1,
        num_workers=0,
        shuffle=False,
        drop_last=False
    )

    # 损失函数与验证指标
    criterion = ContrastLoss() if CLLOSS is True else paddle.nn.loss.L1Loss()
    psnr_compute = PSNR()
    ssim_compute = MS_SSIM()

    # 学习率衰减策略与优化器
    scheduler = paddle.optimizer.lr.CosineAnnealingDecay(learning_rate=LR, T_max=int(TRAIN_SAMPLES_NUMS//PRINT//BS*EPOCH), eta_min=5e-7)
    optimizer = paddle.optimizer.Adam(learning_rate=scheduler, parameters=model.parameters())

    best_score = 0.
    beg = time.time()
    for epoch in range(1, EPOCH+1):
        model.train()
        train_loss = []

        for step, (sample, target) in enumerate(train_loader):
            output = model(sample)
            loss = criterion(output, target, sample) if CLLOSS is True else criterion(output, target)
            train_loss.append(paddle.mean(loss))

            # 按 batch 反向传播
            loss.backward()
            optimizer.step()
            optimizer.clear_gradients()

            # 打印训练损失
            if step % int(PRINT//5) == 0:
                print(f"[{epoch}/{EPOCH}] - Steps {step} - Train loss: {np.mean(train_loss)} - passed: {(time.time() - beg) / 60.}") 

            # 打印验证指标、保存当前最佳模型 (验证时间)
            if (step > 0) and (step % PRINT == 0):  # 
                val_score, val_psnr, val_ssim = valid(valid_loader, model, psnr_compute, ssim_compute)

                print(
                    f"[{epoch}/{EPOCH}] - Steps {step} - Val Score: {val_score}, PSNR: {val_psnr}, SSIM: {val_ssim}, passed: {(time.time() - beg) / 60.}")

                if val_score > best_score:
                    best_score = val_score
                    print(f"======> epoch: {epoch} - step: {step} - best val score: {best_score} <======")
                    
                    # 只有验证指标更优才保存模型
                    paddle.save(model.state_dict(), f'./model_stage{STAGE}_epoch{epoch}_step{step}.pdparams')

                scheduler.step()

            gc.collect()
            paddle.device.cuda.empty_cache()


In [ ]:
# ---------------------------------------------------------------------------------------------------------------------------------
# 主函数调用
# ---------------------------------------------------------------------------------------------------------------------------------

if __name__ == '__main__':
    process()   # 训练！


#### 打包提交




In [1]:
! python ema.py

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
ema finished !!!
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:341: UserWarning: /tmp/tmpd88tptv6.py:8
The behavior of expression A + B has been unified with elementwise_add(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_add(X, Y, axis=0) instead of A + B. This transitional warning will be dropped in the future.
  op_type, op_t

In [2]:
# 压缩提交文件为 submit.zip
! zip submit1.zip model.pdiparams model.pdiparams.info model.pdmodel predict.py

  adding: model.pdiparams (deflated 7%)
  adding: model.pdiparams.info (deflated 82%)
  adding: model.pdmodel (deflated 98%)
  adding: predict.py (deflated 63%)


# 六、模型推理

1. 测试时数据增强

- 循环填充：使用循环填充（右侧&下侧）确保输入样本的宽高为预设尺寸的整数倍，且保持填充前后水印模式一致。
- 等距裁剪分块预测：为避免放缩失真和显存不足，按预设尺寸随机裁剪样本并分别预测，最后将预测结果拼接并移除填充部分。
- 水平翻转自集成：为发挥模型对不同水印模式的泛化能力，将原预测结果和经水平翻转-预测-水平翻转的预测结果等权融合。


![](https://ai-studio-static-online.cdn.bcebos.com/da5d12e330154b009b47ea7165bc2aae69d695fdf607467c8815d1ff7011d0f7)


```

# 原输入样本 shape 为 (n, c, h, w)
n, c, h, w = img.shape

# 将所有图片的尺寸/边长循环 padding 为 SIZE 的倍数
num_patch_h, num_patch_w = (h // patch_size) + 1, (w // patch_size) + 1
# pad_left, pad_right, pad_top, pad_bottom] 
pad = nn.Pad2D(padding=[0, num_patch_w*patch_size-w, 0, num_patch_h*patch_size-h], mode='circular') 
img_pad = pad(img)

# 等距切分为若干尺寸为 (SIZE, SIZE) 的 patch 并分别预测后拼接
pre = np.zeros_like(img_pad.squeeze(0))
with paddle.no_grad():
    for i in range(num_patch_h):
        for j in range(num_patch_w):
            # 当前用于预测的 patch 切片
            img_pad_patch = img_pad[..., i*patch_size: (i+1)*patch_size, j*patch_size: (j+1)*patch_size]
            # 推理期间，验证操作实施等权重水平翻转自集成，以发挥/中和模型对不同水印 pattern 的恢复效果
            pre[..., i*patch_size: (i+1)*patch_size, j*patch_size: (j+1)*patch_size] += (model(img_pad_patch)[0].numpy() * 0.5 + paddle.flip(model(paddle.flip(img_pad_patch, [-1])), [-1])[0].numpy() * 0.5)
                                                                                                
# 切除不需要的预测结果, 还原尺寸为 (h, w) 的原输入样本
pre = pre[..., :h, :w]
```

- 所有推理逻辑均注释于 predict.py 文件。测试时的主要特点在于测试时数据增强 (TTA)。

In [ ]:
! mkdir results
! mkdir testa

In [ ]:
# 解压 测试集A 到 testa 目录下（测试集B同理）
! unzip data/data141714/watermark_testa_datasets.zip -d ./testa

In [ ]:
# 在测试集上预测
! python predict.py ./testa/watermark_test_datasets/images results

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 